In [ ]:
### THIS IS FOR FINAL SUMMARY ###
### Sorted Glycan Forms: for-each-EZ-combination version ###
### ADD EZ DATA ###
### ADD NEW COLUMN MW BY EXTRACTING SEQUENCE-CONTAINED MW NUMBER ###
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib import ticker
from IPython.display import display, HTML
import re # finding specific patterns in str
import textwrap # split text into equal parts
import collections # return repeated items in list
from collections import OrderedDict
import time
from datetime import datetime # attach current date to export filename
import sys
import ast # convert str back to tuple/list/int, etc

# INPUT FILES & SHEET NAMES (ENZYME_N/O GLYCAN format: Ingel_T_C_NG/Ingel_T_C_OG) & EZ_LST (ENZYMES)
filenames = [''
            , ''
            , ''
            ]
sheet_names = ['Ingel_T_C_NG', 'Insol_T_C_NG', 'Insol_aLP_NG', 'All Combined'] # same order as filenames
ez_lst = [i[:-3] for i in sheet_names[:-1]] # format: ['Ingel_T_C', 'Insol_T_C_NG', 'Insol_aLP']

############################### PLEASE DO NOT ALTER ANYTHING BELOW ###############################
# COLUMN NAMES IN DIF INPUT FILE COMBO (2 situations: single/double processing)
if 'Glycans' in df.columns.tolist(): # only b
    glycans_col = 'Glycans'
    sequon_col = 'Sequon'
    site_col = 'N-site(SequonBased)'
    glycantypeanalysis_col = 'GlycanTypeAnalysis'
    psm_col = 'PSM'
    mod_col = 'Mods\r\n(variable)'
elif 'GlycanComposition_ByonicStyle' in df.columns.tolist(): # only p
    glycans_col = 'GlycanComposition_ByonicStyle'
    sequon_col = 'Sequon'
    site_col = 'ProSites'
    glycantypeanalysis_col = 'GlycanTypeAnalysis'
    psm_col = 'PSM'
    mod_col = 'Mod'
elif 'Glycans[Byonic]' in df.columns.tolist() and 'GlycanComposition_ByonicStyle[pGlyco]' not in df.columns.tolist(): # byonic + byos
    glycans_col = 'Glycans[Byonic]'
    sequon_col = 'Sequon[Byonic]'
    site_col = 'N-site(SequonBased)[Byonic]'
    glycantypeanalysis_col = 'GlycanTypeAnalysis[Byonic]'
    psm_col = 'PSM[Byonic]'
    mod_col = 'Mods\r\n(variable)[Byonic]'
else: # bp all present
    b_glycans_col = 'Glycans[Byonic]'
    p_glycans_col = 'GlycanComposition_ByonicStyle[pGlyco]'
    b_sequon_col = 'Sequon[Byonic]'
    p_sequon_col = 'Sequon[pGlyco]'
    b_site_col = 'N-site(SequonBased)[Byonic]'
    p_site_col = 'ProSites[pGlyco]'
    mock_site_col = 'mock_site[Byonic]'
    b_glycantypeanalysis_col = 'GlycanTypeAnalysis[Byonic]'
    p_glycantypeanalysis_col = 'GlycanTypeAnalysis[pGlyco]'
    b_psm_col = 'PSM[Byonic]'
    p_psm_col = 'PSM[pGlyco]'
    b_mod_col = 'Mods\r\n(variable)[Byonic]'
    p_mod_col = 'Mod[pGlyco]'
def SummaryExcel(filenames, sheet_names, ez_lst, software_combo): # software_combo: b, p, bb, bp 
    # TIMESTAMP
    start_time = time.time()
    # RECORD DATE TODAY
    date = datetime.now().strftime('%Y%m%d')
    # DETERMINE EXPORT FILENAME
    export_filename = '_'.join(['_'.join(i.split('_')[:-1]) for i in filenames])
    # MAIN FUNC
    ez_count = 0
    df_lst = []
    gp_df_lst = []
    for filename in filenames: # dif EZ combo
        # FILE PREPROCESSING
        df = pd.read_excel('%s.xlsx'%filename, header = 0)
        df = df.fillna('N/A')
        fixed_colname = [i.replace('_x000D_', '\r') if '_x000D_' in i else i for i in df.columns]
        df.columns = fixed_colname
        # MAIN
        if 'Glycans[Byonic]' in df.columns.tolist() and 'GlycanComposition_ByonicStyle[pGlyco]' in df.columns.tolist(): # bp all present
            b_df = df[[b_site_col, b_sequon_col, b_glycans_col, b_glycantypeanalysis_col, b_mod_col, b_psm_col]]
            p_df = df[[p_site_col, p_sequon_col, p_glycans_col, p_glycantypeanalysis_col, p_mod_col, p_psm_col]]
            b_union = b_df.rename(columns={b_site_col:'N-site(SequonBased)[bpUnion]', b_sequon_col:'Sequon[bpUnion]', b_glycans_col:'Glycans[bpUnion]', b_glycantypeanalysis_col:'GlycanTypeAnalysis[bpUnion]', b_mod_col:'Mod[bpUnion]', b_psm_col:'PSM[bpUnion]'})
            p_union = p_df.rename(columns={p_site_col:'N-site(SequonBased)[bpUnion]', p_sequon_col:'Sequon[bpUnion]', p_glycans_col:'Glycans[bpUnion]', p_glycantypeanalysis_col:'GlycanTypeAnalysis[bpUnion]', p_mod_col:'Mod[bpUnion]', p_psm_col:'PSM[bpUnion]'})
            # delete rows w/ missing sites & drop duplicates
            b_union = b_union.loc[b_union['N-site(SequonBased)[bpUnion]']!=-1].drop_duplicates()
            p_union = p_union.loc[p_union['N-site(SequonBased)[bpUnion]']!=-1].drop_duplicates()
            a1, a2 = b_union.align(p_union, join='outer', axis = 1) # align cols only
            bp_union = pd.concat([a1,a2]).fillna(-1) # concat vertically
            # add EZ col
            bp_union['Enzyme[bpUnion]'] = ez_lst[ez_count]
            # sum PSM (same site/glycan)
            bp_union['sum_PSM[bpUnion]'] = bp_union.groupby(['N-site(SequonBased)[bpUnion]', 'Glycans[bpUnion]'])['PSM[bpUnion]'].transform('sum')
            # sum glycoform (same site/glycoform)
            bp_union['sum_Glycoform[bpUnion]'] = bp_union.groupby(['N-site(SequonBased)[bpUnion]'])['Glycans[bpUnion]'].transform(lambda x: x[x!='N/A'].nunique())
            # gp
            bp_union_gp = bp_union.groupby(['N-site(SequonBased)[bpUnion]', 'Sequon[bpUnion]', 'sum_Glycoform[bpUnion]', 'GlycanTypeAnalysis[bpUnion]', 'Glycans[bpUnion]', 'sum_PSM[bpUnion]', 'Mod[bpUnion]', 'Enzyme[bpUnion]']).agg({'PSM[bpUnion]':'sum'})
            ez_count += 1
            df_lst.append(bp_union) # for summary table (all combined)
            gp_df_lst.append(bp_union_gp) # write to zip
        else: # b/p/bb
            df = df[[site_col, sequon_col, glycans_col, glycantypeanalysis_col, mod_col, psm_col]]
            # add EZ col
            df['Enzyme'] = ez_lst[ez_count]
            # sum PSM (same site/glycan)
            df['sum_PSM'] = df.groupby([site_col, glycans_col])[psm_col].transform('sum')
            # sum glycoform (same site/glycoform)
            df['sum_Glycoform'] = df.groupby([site_col])[glycans_col].transform(lambda x: x[x!='N/A'].nunique())
            # gp
            bp_union_gp = bp_union.groupby([site_col, sequon_col, 'sum_Glycoform', glycantypeanalysis_col, glycans_col, 'sum_PSM', mod_col, 'Enzyme']).agg({psm_col:'sum'})
            ez_count += 1
            df_lst.append(bp_union) # for summary table (all combined)
            gp_df_lst.append(bp_union_gp) # write to zip
           
    # CONCAT DIF EZ COMBO FOR FINAL SUMMARY
    all_ez_combo = pd.concat(df_lst)
    if 'Glycans[Byonic]' in df.columns.tolist() and 'GlycanComposition_ByonicStyle[pGlyco]' in df.columns.tolist(): # bp all present
        # sum PSM (same site/glycan) again
        all_ez_combo['sum_PSM[bpUnion]'] = all_ez_combo.groupby(['N-site(SequonBased)[bpUnion]', 'Glycans[bpUnion]'])['PSM[bpUnion]'].transform('sum')
        # sum glycoform (same site/glycoform) again
        all_ez_combo['sum_Glycoform[bpUnion]'] = all_ez_combo.groupby(['N-site(SequonBased)[bpUnion]'])['Glycans[bpUnion]'].transform(lambda x: x[x!='N/A'].nunique())
        # gp again
        all_ez_combo_gp = all_ez_combo.groupby(['N-site(SequonBased)[bpUnion]', 'Sequon[bpUnion]', 'sum_Glycoform[bpUnion]', 'GlycanTypeAnalysis[bpUnion]', 'Glycans[bpUnion]', 'sum_PSM[bpUnion]', 'Mod[bpUnion]']).agg({'Enzyme[bpUnion]': lambda x : '/ '.join(x), 'PSM[bpUnion]':'sum'})
        gp_df_lst.append(all_ez_combo_gp)
    else: # b/p/bb
        # sum PSM (same site/glycan) again
        all_ez_combo['sum_PSM'] = all_ez_combo.groupby([site_col, glycans_col])[psm_col].transform('sum')
        # sum glycoform (same site/glycoform) again
        all_ez_combo['sum_Glycoform'] = all_ez_combo.groupby([site_col])[glycans_col].transform(lambda x: x[x!='N/A'].nunique())
        # gp again
        all_ez_combo_gp = all_ez_combo.groupby([site_col, sequon_col, 'sum_Glycoform', glycantypeanalysis_col, glycans_col, 'sum_PSM', mod_col]).agg({'Enzyme': lambda x : '/ '.join(x), psm_col:'sum'})
        gp_df_lst.append(all_ez_combo_gp)
    
    # WRITE ALL GP DF TO ZIP 
    sheet_name_count = 0
    with pd.ExcelWriter(f'{date}_Summary_{export_filename}.xlsx') as writer:
        for df in gp_df_lst:
            df.style.background_gradient(cmap ='Reds').to_excel(writer, sheet_name = sheet_names[sheet_name_count])
            sheet_name_count += 1
    return all_ez_combo_gp

# START PLOTTING SUMMARY DF
def Plotter(df, glycantypecol, nsite_col, psm_col, software=''): # HERE THE DF IS GP_DF
    site_nostr_new = list(sorted(set([ast.literal_eval(i) if type(i)==str else i for i in df.index.get_level_values(0).tolist()]), key=[ast.literal_eval(i) if type(i)==str else i for i in df.index.get_level_values(0).tolist()].index))
    df.index.set_levels(site_nostr_new, level = 0, inplace=True)
    site_nostr_new = sorted(list(set(site_nostr_new)), key=lambda v: (isinstance(v, tuple), v))
    site_nostr_new = sorted(site_nostr_new, key=lambda x: x[0] if type(x)==tuple else x)
    # sort site by new order
    df = df.reindex(site_nostr_new, level = 0)
    # reset index to un-group the df
    df = df.reset_index()
#     CONTINUE WRITING FROM HERE!!!
    # prepare key var
    glycosylation_site = df[nsite_col].tolist()
    glycan_analysis_result = df[glycantypecol].tolist()
    psm = df[psm_col].tolist()

    bar_size = [] #length == site number.
    pie_size = [] #length == site number.
    for site in site_nostr_new: #loop thru non-repeated sites.
        print(site)
        each_bar_size = [] #length == 16.
        each_pie_size = [] #length == 4.
        man9, man8, man7, man6, man5, hybrid, fhybrid, a1, fa1, a2a1b, fa2fa1b, a3a2b, fa3fa2b, a4a3b, fa4fa3b, unoccupied, multi_highman, multi_complex_hybrid = 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
        each_site_ind = [pos for pos, char in enumerate(glycosylation_site) if char == site]
        print('each_site_ind:%s\n'%each_site_ind)
        each_site_ind_cnt = 0
        each_glycan_type = [glycan_analysis_result[i] for i in each_site_ind] #all the glycan analysis results within one site (a list).
        print('each_glycan_type:%s'%each_glycan_type)
        for glycan in each_glycan_type:
            print('this is glycan: %s'%glycan)
            if glycan == 'Man9':
                man9 += psm[each_site_ind[each_site_ind_cnt]]
                each_site_ind_cnt += 1 
            elif glycan == 'Man8':
                man8 += psm[each_site_ind[each_site_ind_cnt]]
                each_site_ind_cnt += 1
            elif glycan == 'Man7':
                man7 += psm[each_site_ind[each_site_ind_cnt]]
                each_site_ind_cnt += 1
            elif glycan == 'Man6':
                man6 += psm[each_site_ind[each_site_ind_cnt]]
                each_site_ind_cnt += 1
            elif glycan == 'Man5':
                man5 += psm[each_site_ind[each_site_ind_cnt]]
                each_site_ind_cnt += 1
            elif glycan == 'Man4': # do not plot man4
                each_site_ind_cnt += 1
            elif glycan == 'Hybrid':
                hybrid += psm[each_site_ind[each_site_ind_cnt]]
                each_site_ind_cnt += 1
            elif glycan == 'Fhybrid':
                fhybrid += psm[each_site_ind[each_site_ind_cnt]]
                each_site_ind_cnt += 1
            elif glycan == 'A1':
                a1 += psm[each_site_ind[each_site_ind_cnt]]
                each_site_ind_cnt += 1
            elif glycan == 'FA1':
                fa1 += psm[each_site_ind[each_site_ind_cnt]]
                each_site_ind_cnt += 1
            elif glycan == 'A2/A1B':
                a2a1b += psm[each_site_ind[each_site_ind_cnt]]
                each_site_ind_cnt += 1
            elif glycan == 'FA2/FA1B':
                fa2fa1b += psm[each_site_ind[each_site_ind_cnt]]
                each_site_ind_cnt += 1
            elif glycan == 'A3/A2B':
                a3a2b += psm[each_site_ind[each_site_ind_cnt]]
                each_site_ind_cnt += 1
            elif glycan == 'FA3/FA2B':
                fa3fa2b += psm[each_site_ind[each_site_ind_cnt]]
                each_site_ind_cnt += 1
            elif glycan == 'A4/A3B':
                a4a3b += psm[each_site_ind[each_site_ind_cnt]]
                each_site_ind_cnt += 1
            elif glycan == 'FA4/FA3B':
                fa4fa3b += psm[each_site_ind[each_site_ind_cnt]]
                each_site_ind_cnt += 1
            elif glycan == 'The core is not complete!':
                each_site_ind_cnt += 1
            elif glycan == 'Only core!':
                each_site_ind_cnt += 1
            elif glycan == 'Potential N-glycan precursor':
                each_site_ind_cnt += 1
            elif glycan == 'Multi Site: Only 1 core!':
                each_site_ind_cnt += 1
            elif glycan == 'Multi Site: Cannot form any core!':
                each_site_ind_cnt += 1
            elif glycan == 'Multi Site [count PSM once]: Highmannose':
                multi_highman += psm[each_site_ind[each_site_ind_cnt]] 
                each_site_ind_cnt += 1
            elif glycan == 'Multi Site [count PSM once]: Hybrid/Complex':
                multi_complex_hybrid += psm[each_site_ind[each_site_ind_cnt]]
                each_site_ind_cnt += 1
            elif glycan == 'Multi Site: 2 cores!':
                each_site_ind_cnt += 1
            elif glycan == 'Multi Site [count PSM twice]: Highmannose':
                multi_highman += psm[each_site_ind[each_site_ind_cnt]]*2
                each_site_ind_cnt += 1
                print('multi_highman:%s\n'%multi_highman)
            elif glycan == 'Multi Site [count PSM twice]: Hybrid/Complex':
                multi_complex_hybrid += psm[each_site_ind[each_site_ind_cnt]]*2
                each_site_ind_cnt += 1
            elif glycan == 'Unoccupied': 
                unoccupied += psm[each_site_ind[each_site_ind_cnt]]
                each_site_ind_cnt += 1
            print('final multi_highman:%s\n'%multi_highman)
        if type(site) == tuple: # multi-sites (more than one sequon)
            each_bar_size.extend([multi_highman, multi_complex_hybrid, unoccupied])
            bar_size.append(each_bar_size)
            Highman = each_bar_size[0]
            Hybrid_Complex = each_bar_size[1]
            Unoccupied = each_bar_size[-1]
            each_pie_size.extend([Highman, Hybrid_Complex, Unoccupied])
            pie_size.append(each_pie_size)
        else: # real single sites (single sequon)
            each_bar_size.extend([man9, man8, man7, man6, man5, hybrid, fhybrid, a1, fa1, a2a1b, fa2fa1b, a3a2b, fa3fa2b, a4a3b, fa4fa3b, unoccupied])
            bar_size.append(each_bar_size)
            Highman = sum(each_bar_size[:5])
            Hybrid = sum(each_bar_size[5:7])
            Complex = sum(each_bar_size[7:15])
            Unoccupied = each_bar_size[-1]
            each_pie_size.extend([Highman, Hybrid, Complex, Unoccupied])
            pie_size.append(each_pie_size)
    print('this is bar_size:\n%s'%bar_size)
    original_bar_size = bar_size #save for later manipulation. 
    print('this is pie_size:\n%s'%pie_size)
    original_pie_size = pie_size #save for later manipulation.
    print('length of the bar_size (also the final site number): %s'%len(bar_size))
    print('length of the pie_size (also the final site number): %s'%len(pie_size))

    ##record glycosylation site indices.
    sites_for_plot = ['N%s'%site if type(site)==int else '/\n'.join(['N%s'%(site[i]) for i in range(len(site))]) for site in site_nostr_new]
    pie_types = ['HighMannose', 'Hybrid', 'Complex', 'Hybrid/Complex', 'Unoccupied']
    bar_types = ['Man9', 'Man8', 'Man7', 'Man6', 'Man5', 'Hybrid', 'Fhybrid', 'A1', 'FA1', 'A2/A1B', 'FA2/FA1B', 'A3/A2B', 'FA3/FA2B', 'A4/A3B', 'FA4/FA3B', 'Unoccupied']
    multi_bar_types = ['HighMannose', 'Hybrid/Complex', 'Unoccupied']
    colors = ['#008000', '#ffffff','#ff69b4','#d3d3d3'] 
    multi_colors = ['#008000', '#ffc0cb', '#d3d3d3']
    leg_colors = ['#008000', '#ffffff','#ff69b4', '#ffc0cb', '#d3d3d3']

    ##start plotting.
    ## PLOT BAR & PIE COMBINED CHARTS FIRST.
    ##### adjustable parameters ###################
    if len(bar_size) <= 4:
        number_of_sites_inrow = 1
        full_row_num = len(bar_size)//number_of_sites_inrow
        remainder_num = len(bar_size)%number_of_sites_inrow
        bar_remainder = bar_size[-remainder_num:] #last ones.
        print('bar_remainder: %s'%bar_remainder)
        bar_size = [bar_size[number_of_sites_inrow*i:number_of_sites_inrow*(i+1)] for i in range(full_row_num)]
        pie_remainder = pie_size[-remainder_num:] #last ones.
        print('pie_remainder: %s'%pie_remainder)
        pie_size = [pie_size[number_of_sites_inrow*i:number_of_sites_inrow*(i+1)] for i in range(full_row_num)]
        print('this is resized bar_size: %s'%bar_size)
        print('this is resized pie_size: %s'%pie_size)
    else:
        number_of_sites_inrow = 4
        full_row_num = len(bar_size)//number_of_sites_inrow
        remainder_num = len(bar_size)%number_of_sites_inrow
        bar_remainder = bar_size[-remainder_num:] #last ones.
        print('bar_remainder: %s'%bar_remainder)
        bar_size = [bar_size[number_of_sites_inrow*i:number_of_sites_inrow*(i+1)] for i in range(full_row_num)]
        bar_size.append(bar_remainder)
        pie_remainder = pie_size[-remainder_num:] #last ones.
        print('pie_remainder: %s'%pie_remainder)
        pie_size = [pie_size[number_of_sites_inrow*i:number_of_sites_inrow*(i+1)] for i in range(full_row_num)]
        pie_size.append(pie_remainder)
        print('this is resized bar_size: %s'%bar_size)
        print('this is resized pie_size: %s'%pie_size)

    if remainder_num == 0:
        row_num = full_row_num
    else: 
        row_num = full_row_num + 1

    print('this is row_num: %s'%row_num)
    print('this is remainder_num: %s'%remainder_num)
    col_num = 2*number_of_sites_inrow
    fig_ratio = number_of_sites_inrow/row_num
    ###############################################

    #make everything in the plot bold & basic setups.
    ####### fontsize & spine width #######
    bar_title_fontsize = 12
    bar_title_ha = 'left'
    SUBPLOT_PAD = 10
    bar_ylab_fontsize = bar_title_fontsize - 4
    bar_width = 0.7
    multi_bar_width = 0.1
    x_tick_fontsize = bar_title_fontsize -4 
    y_tick_fonsize = bar_title_fontsize -4
    x_tick_w = 1.2
    x_tick_l = x_tick_w*3
    y_tick_w = 1.2
    y_tick_l = y_tick_w*3
    spine_width = 1.2
    pie_line_w = 1.2
    bar_line_w = pie_line_w
    legend_fontsize = bar_title_fontsize
    legend_handleL = 2
    MULTI_SITE_BAR_FONT_SIZE = 8
    MULTI_SITE_BAR_LABEL_PAD = 0
    PSM_LABEL_PAD = 1.5
    PIE_R = 1.75
    HATCH = '/'*4
    TICKER_THRESHOLD = 6
    MAXNLOCATOR_NBINS = 5
    COMBINED_PLOTS_RATIO_1 = 5
    COMBINED_PLOTS_RATIO_2 = 1.5
    ######################################
    plt.rcParams["font.weight"] = "bold"
    plt.rcParams["axes.labelweight"] = "bold"
    fig, axs = plt.subplots(nrows=row_num, ncols=col_num, figsize=(COMBINED_PLOTS_RATIO_1*number_of_sites_inrow, COMBINED_PLOTS_RATIO_2*row_num), constrained_layout=True)
    site_name_count = 0

    if row_num >1: 
        for i in range(row_num):
            if i != (row_num -1): #other than the last row. 
                for j in range(number_of_sites_inrow): #N sites from left to right.
                    if max(bar_size[i][j]) > TICKER_THRESHOLD: #use ticker.
                        if '/' in sites_for_plot[site_name_count]: #multi-sites.
                            #pie
                            axs[i, 2*j+1].pie(pie_size[i][j], radius=PIE_R, colors = multi_colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})
                            axs[i, 2*j+1].pie(pie_size[i][j], radius=PIE_R, colors = multi_colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})[0][1].set_hatch(HATCH)
                            #bar
                            axs[i, 2*j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                            axs[i, 2*j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                            yticks = ticker.MaxNLocator(MAXNLOCATOR_NBINS, integer = True)
                            axs[i, 2*j].yaxis.set_major_locator(yticks)
                            for axis in ["left", 'bottom']:
                                axs[i, 2*j].spines[axis].set_linewidth(spine_width)
                            for spine in ["top", "right"]:
                                axs[i, 2*j].spines[spine].set_visible(False)
                            index = [i for i in range(3)]
                            xticks = np.arange(3)
                            #yticks = np.arange(max(size[i])+1)
                            axs[i, 2*j].bar(index[0], bar_size[i][j][0], multi_bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                            axs[i, 2*j].bar(index[1], bar_size[i][j][1], multi_bar_width, color=['#ffc0cb'], edgecolor='black', linewidth = bar_line_w, hatch=HATCH)
                            axs[i, 2*j].bar(index[2], bar_size[i][j][2], multi_bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                            axs[i, 2*j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                            axs[i, 2*j].title.set_position([-0.15, 1.1])
                            axs[i, 2*j].set_xticks(xticks, minor=False)
                            axs[i, 2*j].set_xticklabels('')
                            #axs[i, 2*j].set_xticklabels(multi_bar_types , fontweight="bold" ,minor=False, rotation = 45, ha = 'center', fontsize = x_tick_fontsize)
                            #ax1.set_yticks(yticks, minor=False)
                            #ax1.set_ylim(yticks[0], yticks[-1])
                            axs[i, 2*j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                            bars = [rect for rect in axs[i, 2*j].get_children() if isinstance(rect, mpl.patches.Rectangle)]
                            print('this is the length of bars: %s'%len(bars))
                            bar_type_count = 0
                            multi_bar_labels = ['HM', 'H/C', 'UN']
                            for bar in bars:
                                if bar_type_count != 3: 
                                    height = bar.get_height() + MULTI_SITE_BAR_LABEL_PAD
                                    label_x_pos = bar.get_x() + bar.get_width() / 2
                                    axs[i, 2*j].text(label_x_pos, height, s=multi_bar_labels[bar_type_count], ha='center',
                                    va='bottom', fontsize = MULTI_SITE_BAR_FONT_SIZE)
                                    bar_type_count += 1
                                else:
                                    pass
                            site_name_count += 1
                        else: #real singel sites.
                            #pie
                            axs[i, 2*j+1].pie(pie_size[i][j], radius=PIE_R, colors = colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})
                            #bar
                            axs[i, 2*j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                            axs[i, 2*j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                            yticks = ticker.MaxNLocator(MAXNLOCATOR_NBINS, integer = True)
                            axs[i, 2*j].yaxis.set_major_locator(yticks)
                            for axis in ["left", 'bottom']:
                                axs[i, 2*j].spines[axis].set_linewidth(spine_width)
                            for spine in ["top", "right"]:
                                axs[i, 2*j].spines[spine].set_visible(False)
                            index = [i for i in range(16)]
                            xticks = np.arange(16)
                            #yticks = np.arange(max(size[i])+1)
                            axs[i, 2*j].bar(index[0:5], bar_size[i][j][0:5], bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                            axs[i, 2*j].bar(index[5:7], bar_size[i][j][5:7], bar_width, color=['#ffffff'], edgecolor='black', linewidth = bar_line_w)
                            axs[i, 2*j].bar(index[7:15], bar_size[i][j][7:15], bar_width, color=['#ff69b4'], edgecolor='black', linewidth = bar_line_w)
                            axs[i, 2*j].bar(index[15], bar_size[i][j][15], bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                            axs[i, 2*j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                            axs[i, 2*j].title.set_position([-0.15, 1.1])
                            axs[i, 2*j].set_xticks(xticks, minor=False)
                            axs[i, 2*j].set_xticklabels('')
                            #ax1.set_yticks(yticks, minor=False)
                            #ax1.set_ylim(yticks[0], yticks[-1])
                            axs[i, 2*j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                            site_name_count += 1
                    else: #do not use ticker.
                        if '/' in sites_for_plot[site_name_count]: #multi-sites.
                            #pie
                            axs[i, 2*j+1].pie(pie_size[i][j], radius=PIE_R, colors = multi_colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})
                            axs[i, 2*j+1].pie(pie_size[i][j], radius=PIE_R, colors = multi_colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})[0][1].set_hatch(HATCH)
                            #bar
                            axs[i, 2*j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                            axs[i, 2*j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                            #yticks = ticker.MaxNLocator(6)
                            #axs[i, 2*j].yaxis.set_major_locator(yticks)
                            for axis in ["left", 'bottom']:
                                axs[i, 2*j].spines[axis].set_linewidth(spine_width)
                            for spine in ["top", "right"]:
                                axs[i, 2*j].spines[spine].set_visible(False)
                            index = [i for i in range(3)]
                            xticks = np.arange(3)
                            yticks = np.arange(max(bar_size[i][j])+1)
                            axs[i, 2*j].bar(index[0], bar_size[i][j][0], multi_bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                            axs[i, 2*j].bar(index[1], bar_size[i][j][1], multi_bar_width, color=['#ffc0cb'], edgecolor='black', linewidth = bar_line_w, hatch=HATCH)
                            axs[i, 2*j].bar(index[2], bar_size[i][j][2], multi_bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                            axs[i, 2*j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                            axs[i, 2*j].title.set_position([-0.15, 1.1])
                            axs[i, 2*j].set_xticks(xticks, minor=False)
                            axs[i, 2*j].set_xticklabels('')
                            #axs[i, 2*j].set_xticklabels(multi_bar_types , fontweight="bold" ,minor=False, rotation = 45, ha = 'center', fontsize = x_tick_fontsize)
                            axs[i, 2*j].set_yticks(yticks, minor=False)
                            axs[i, 2*j].set_ylim(yticks[0], yticks[-1])
                            axs[i, 2*j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                            bars = [rect for rect in axs[i, 2*j].get_children() if isinstance(rect, mpl.patches.Rectangle)]
                            print('this is the length of bars: %s'%len(bars))
                            bar_type_count = 0
                            multi_bar_labels = ['HM', 'H/C', 'UN']
                            for bar in bars:
                                if bar_type_count != 3: 
                                    height = bar.get_height() + MULTI_SITE_BAR_LABEL_PAD 
                                    label_x_pos = bar.get_x() + bar.get_width() / 2
                                    axs[i, 2*j].text(label_x_pos, height, s=multi_bar_labels[bar_type_count], ha='center',
                                    va='bottom', fontsize = MULTI_SITE_BAR_FONT_SIZE)
                                    bar_type_count += 1
                                else:
                                    pass
                            site_name_count += 1
                        else: #real singel sites.
                            #pie
                            axs[i, 2*j+1].pie(pie_size[i][j], radius=PIE_R, colors = colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})
                            #bar
                            axs[i, 2*j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                            axs[i, 2*j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                            #yticks = ticker.MaxNLocator(6)
                            #axs[i, 2*j].yaxis.set_major_locator(yticks)
                            for axis in ["left", 'bottom']:
                                axs[i, 2*j].spines[axis].set_linewidth(spine_width)
                            for spine in ["top", "right"]:
                                axs[i, 2*j].spines[spine].set_visible(False)
                            index = [i for i in range(16)]
                            xticks = np.arange(16)
                            yticks = np.arange(max(bar_size[i][j])+1)
                            axs[i, 2*j].bar(index[0:5], bar_size[i][j][0:5], bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                            axs[i, 2*j].bar(index[5:7], bar_size[i][j][5:7], bar_width, color=['#ffffff'], edgecolor='black', linewidth = bar_line_w)
                            axs[i, 2*j].bar(index[7:15], bar_size[i][j][7:15], bar_width, color=['#ff69b4'], edgecolor='black', linewidth = bar_line_w)
                            axs[i, 2*j].bar(index[15], bar_size[i][j][15], bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                            axs[i, 2*j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                            axs[i, 2*j].title.set_position([-0.15, 1.1])
                            axs[i, 2*j].set_xticks(xticks, minor=False)
                            axs[i, 2*j].set_xticklabels('')
                            axs[i, 2*j].set_yticks(yticks, minor=False)
                            axs[i, 2*j].set_ylim(yticks[0], yticks[-1])
                            axs[i, 2*j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                            site_name_count += 1
            else: #for the last row. Deal with the remainder.
                if remainder_num == 0: #can be divided by 4.
                    last_row_site_num = number_of_sites_inrow
                else: #cannot be divided by 4.
                    last_row_site_num = remainder_num
                print('this is last_row_site_num: %s'%last_row_site_num)
                for j in range(number_of_sites_inrow):
                    if j < last_row_site_num:  
                        print('this is i,j: %s,%s'%(i,j))
                        if max(bar_size[i][j]) > TICKER_THRESHOLD: #use ticker.
                            if '/' in sites_for_plot[site_name_count]: #multi-sites.
                                #pie
                                axs[i, 2*j+1].pie(pie_size[i][j], radius=PIE_R, colors = multi_colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})
                                axs[i, 2*j+1].pie(pie_size[i][j], radius=PIE_R, colors = multi_colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})[0][1].set_hatch(HATCH)
                                #bar
                                axs[i, 2*j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                                axs[i, 2*j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                                yticks = ticker.MaxNLocator(MAXNLOCATOR_NBINS, integer = True)
                                axs[i, 2*j].yaxis.set_major_locator(yticks)
                                for axis in ["left", 'bottom']:
                                    axs[i, 2*j].spines[axis].set_linewidth(spine_width)
                                for spine in ["top", "right"]:
                                    axs[i, 2*j].spines[spine].set_visible(False)
                                index = [i for i in range(3)]
                                xticks = np.arange(3)
                                #yticks = np.arange(max(size[i])+1)
                                axs[i, 2*j].bar(index[0], bar_size[i][j][0], multi_bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, 2*j].bar(index[1], bar_size[i][j][1], multi_bar_width, color=['#ffc0cb'], edgecolor='black', linewidth = bar_line_w, hatch=HATCH)
                                axs[i, 2*j].bar(index[2], bar_size[i][j][2], multi_bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, 2*j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                                axs[i, 2*j].title.set_position([-0.15, 1.1])
                                axs[i, 2*j].set_xticks(xticks, minor=False)
                                axs[i, 2*j].set_xticklabels(multi_bar_types , fontweight="bold" ,minor=False, rotation = 45, ha = 'right', fontsize = x_tick_fontsize)
                                #ax1.set_yticks(yticks, minor=False)
                                #ax1.set_ylim(yticks[0], yticks[-1])
                                axs[i, 2*j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                                bars = [rect for rect in axs[i, 2*j].get_children() if isinstance(rect, mpl.patches.Rectangle)]
                                print('this is the length of bars: %s'%len(bars))
                                bar_type_count = 0
                                multi_bar_labels = ['HM', 'H/C', 'UN']
                                for bar in bars:
                                    if bar_type_count != 3: 
                                        height = bar.get_height() + MULTI_SITE_BAR_LABEL_PAD
                                        label_x_pos = bar.get_x() + bar.get_width() / 2
                                        axs[i, 2*j].text(label_x_pos, height, s=multi_bar_labels[bar_type_count], ha='center',
                                        va='bottom', fontsize = MULTI_SITE_BAR_FONT_SIZE)
                                        bar_type_count += 1
                                    else:
                                        pass
                                site_name_count += 1
                            else: #real single sites.
                                #pie
                                axs[i, 2*j+1].pie(pie_size[i][j], radius=PIE_R, colors = colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})
                                #bar
                                axs[i, 2*j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                                axs[i, 2*j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                                yticks = ticker.MaxNLocator(MAXNLOCATOR_NBINS, integer = True)
                                axs[i, 2*j].yaxis.set_major_locator(yticks)
                                for axis in ["left", 'bottom']:
                                    axs[i, 2*j].spines[axis].set_linewidth(spine_width)
                                for spine in ["top", "right"]:
                                    axs[i, 2*j].spines[spine].set_visible(False)
                                index = [i for i in range(16)]
                                xticks = np.arange(16)
                                #yticks = np.arange(max(size[i])+1)
                                axs[i, 2*j].bar(index[0:5], bar_size[i][j][0:5], bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, 2*j].bar(index[5:7], bar_size[i][j][5:7], bar_width, color=['#ffffff'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, 2*j].bar(index[7:15], bar_size[i][j][7:15], bar_width, color=['#ff69b4'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, 2*j].bar(index[15], bar_size[i][j][15], bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, 2*j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                                axs[i, 2*j].title.set_position([-0.15, 1.1])
                                axs[i, 2*j].set_xticks(xticks, minor=False)
                                axs[i, 2*j].set_xticklabels( bar_types, fontweight="bold" ,minor=False, rotation = 90, ha = 'center', fontsize = x_tick_fontsize)
                                #ax1.set_yticks(yticks, minor=False)
                                #ax1.set_ylim(yticks[0], yticks[-1])
                                axs[i, 2*j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                                site_name_count += 1
                        else: #do not use ticker.
                            if '/' in sites_for_plot[site_name_count]: #multi-sites.
                                #pie
                                axs[i, 2*j+1].pie(pie_size[i][j], radius=PIE_R, colors = multi_colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})
                                axs[i, 2*j+1].pie(pie_size[i][j], radius=PIE_R, colors = multi_colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})[0][1].set_hatch(HATCH)
                                #bar
                                axs[i, 2*j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                                axs[i, 2*j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                                #yticks = ticker.MaxNLocator(6)
                                #axs[i, 2*j].yaxis.set_major_locator(yticks)
                                for axis in ["left", 'bottom']:
                                    axs[i, 2*j].spines[axis].set_linewidth(spine_width)
                                for spine in ["top", "right"]:
                                    axs[i, 2*j].spines[spine].set_visible(False)
                                index = [i for i in range(3)]
                                xticks = np.arange(3)
                                yticks = np.arange(max(bar_size[i][j])+1)
                                axs[i, 2*j].bar(index[0], bar_size[i][j][0], multi_bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, 2*j].bar(index[1], bar_size[i][j][1], multi_bar_width, color=['#ffc0cb'], edgecolor='black', linewidth = bar_line_w, hatch=HATCH)
                                axs[i, 2*j].bar(index[2], bar_size[i][j][2], multi_bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, 2*j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                                axs[i, 2*j].title.set_position([-0.15, 1.1])
                                axs[i, 2*j].set_xticks(xticks, minor=False)
                                axs[i, 2*j].set_xticklabels(multi_bar_types , fontweight="bold" ,minor=False, rotation = 45, ha = 'right', fontsize = x_tick_fontsize)
                                axs[i, 2*j].set_yticks(yticks, minor=False)
                                axs[i, 2*j].set_ylim(yticks[0], yticks[-1])
                                axs[i, 2*j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                                bars = [rect for rect in axs[i, 2*j].get_children() if isinstance(rect, mpl.patches.Rectangle)]
                                print('this is the length of bars: %s'%len(bars))
                                bar_type_count = 0
                                multi_bar_labels = ['HM', 'H/C', 'UN']
                                for bar in bars:
                                    if bar_type_count != 3: 
                                        height = bar.get_height() + MULTI_SITE_BAR_LABEL_PAD
                                        label_x_pos = bar.get_x() + bar.get_width() / 2
                                        axs[i, 2*j].text(label_x_pos, height, s=multi_bar_labels[bar_type_count], ha='center',
                                        va='bottom', fontsize = MULTI_SITE_BAR_FONT_SIZE)
                                        bar_type_count += 1
                                    else:
                                        pass
                                site_name_count += 1
                            else: #real single sites.
                                print('this is i,j: %s,%s'%(i,j))
                                #pie
                                axs[i, 2*j+1].pie(pie_size[i][j], radius=PIE_R, colors = colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})
                                #bar
                                axs[i, 2*j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                                axs[i, 2*j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                                #yticks = ticker.MaxNLocator(6)
                                #axs[i, 0].yaxis.set_major_locator(yticks)
                                for axis in ["left", 'bottom']:
                                    axs[i, 2*j].spines[axis].set_linewidth(spine_width)
                                for spine in ["top", "right"]:
                                    axs[i, 2*j].spines[spine].set_visible(False)
                                index = [i for i in range(16)]
                                xticks = np.arange(16)
                                yticks = np.arange(max(bar_size[i][j])+1)
                                axs[i, 2*j].bar(index[0:5], bar_size[i][j][0:5], bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, 2*j].bar(index[5:7], bar_size[i][j][5:7], bar_width, color=['#ffffff'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, 2*j].bar(index[7:15], bar_size[i][j][7:15], bar_width, color=['#ff69b4'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, 2*j].bar(index[15], bar_size[i][j][15], bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, 2*j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                                axs[i, 2*j].title.set_position([-0.15, 1.1])
                                axs[i, 2*j].set_xticks(xticks, minor=False)
                                axs[i, 2*j].set_xticklabels(bar_types , fontweight="bold" ,minor=False, rotation = 90, ha = 'center', fontsize = x_tick_fontsize)
                                axs[i, 2*j].set_yticks(yticks, minor=False)
                                axs[i, 2*j].set_ylim(yticks[0], yticks[-1])
                                axs[i, 2*j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                                site_name_count += 1
                    else: #empty subplot(s). hide spines & ticks & labels.
                        print('this is i,j for empty subplots: %s,%s'%(i,j))
                        axs[i, 2*j].spines["left"].set_visible(False)
                        axs[i, 2*j].spines["right"].set_visible(False)
                        axs[i, 2*j].spines["top"].set_visible(False)
                        axs[i, 2*j].spines["bottom"].set_visible(False)
                        axs[i, 2*j+1].spines["left"].set_visible(False)
                        axs[i, 2*j+1].spines["right"].set_visible(False)
                        axs[i, 2*j+1].spines["top"].set_visible(False)
                        axs[i, 2*j+1].spines["bottom"].set_visible(False)
                        axs[i, 2*j].xaxis.set_ticks([])
                        axs[i, 2*j].yaxis.set_ticks([])
                        axs[i, 2*j+1].xaxis.set_ticks([])
                        axs[i, 2*j+1].yaxis.set_ticks([])
    else: #if only one row. (based on the layout setting above, this will happen if there's only one site.)
        i = 0
        if remainder_num == 0: #can be divided by 4.
            last_row_site_num = number_of_sites_inrow
        else: #cannot be divided by 4.
            last_row_site_num = remainder_num
        print('this is last_row_site_num: %s'%last_row_site_num)
        for j in range(number_of_sites_inrow):
            if j < last_row_site_num:  
                if max(bar_size[i][j]) > TICKER_THRESHOLD: #use ticker.
                    if '/' in sites_for_plot[site_name_count]: #multi-sites.
                        #pie
                        axs[2*j+1].pie(pie_size[i][j], radius=PIE_R, colors = multi_colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})
                        axs[2*j+1].pie(pie_size[i][j], radius=PIE_R, colors = multi_colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})[0][1].set_hatch(HATCH)
                        #bar
                        axs[2*j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                        axs[2*j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                        yticks = ticker.MaxNLocator(MAXNLOCATOR_NBINS, integer = True)
                        axs[2*j].yaxis.set_major_locator(yticks)
                        for axis in ["left", 'bottom']:
                            axs[2*j].spines[axis].set_linewidth(spine_width)
                        for spine in ["top", "right"]:
                            axs[2*j].spines[spine].set_visible(False)
                        index = [i for i in range(3)]
                        xticks = np.arange(3)
                        #yticks = np.arange(max(size[i])+1)
                        axs[2*j].bar(index[0], bar_size[i][j][0], multi_bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                        axs[2*j].bar(index[1], bar_size[i][j][1], multi_bar_width, color=['#ffc0cb'], edgecolor='black', linewidth = bar_line_w, hatch=HATCH)
                        axs[2*j].bar(index[2], bar_size[i][j][2], multi_bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                        axs[2*j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                        axs[2*j].title.set_position([-0.15, 1.1])
                        axs[2*j].set_xticks(xticks, minor=False)
                        axs[2*j].set_xticklabels(multi_bar_types , fontweight="bold" ,minor=False, rotation = 45, ha = 'right', fontsize = x_tick_fontsize)
                        #ax1.set_yticks(yticks, minor=False)
                        #ax1.set_ylim(yticks[0], yticks[-1])
                        axs[2*j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                        bars = [rect for rect in axs[2*j].get_children() if isinstance(rect, mpl.patches.Rectangle)]
                        print('this is the length of bars: %s'%len(bars))
                        bar_type_count = 0
                        multi_bar_labels = ['HM', 'H/C', 'UN']
                        for bar in bars:
                            if bar_type_count != 3: 
                                height = bar.get_height() + MULTI_SITE_BAR_LABEL_PAD
                                label_x_pos = bar.get_x() + bar.get_width() / 2
                                axs[2*j].text(label_x_pos, height, s=multi_bar_labels[bar_type_count], ha='center',
                                va='bottom', fontsize = MULTI_SITE_BAR_FONT_SIZE)
                                bar_type_count += 1
                            else:
                                pass
                        site_name_count += 1
                    else: #real single sites.
                        #pie
                        axs[2*j+1].pie(pie_size[i][j], radius=PIE_R, colors = colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})
                        #bar
                        axs[2*j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                        axs[2*j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                        yticks = ticker.MaxNLocator(MAXNLOCATOR_NBINS, integer = True)
                        axs[2*j].yaxis.set_major_locator(yticks)
                        for axis in ["left", 'bottom']:
                            axs[2*j].spines[axis].set_linewidth(spine_width)
                        for spine in ["top", "right"]:
                            axs[2*j].spines[spine].set_visible(False)
                        index = [i for i in range(16)]
                        xticks = np.arange(16)
                        #yticks = np.arange(max(size[i])+1)
                        axs[2*j].bar(index[0:5], bar_size[i][j][0:5], bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                        axs[2*j].bar(index[5:7], bar_size[i][j][5:7], bar_width, color=['#ffffff'], edgecolor='black', linewidth = bar_line_w)
                        axs[2*j].bar(index[7:15], bar_size[i][j][7:15], bar_width, color=['#ff69b4'], edgecolor='black', linewidth = bar_line_w)
                        axs[2*j].bar(index[15], bar_size[i][j][15], bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                        axs[2*j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                        axs[2*j].title.set_position([-0.15, 1.1])
                        axs[2*j].set_xticks(xticks, minor=False)
                        axs[2*j].set_xticklabels( bar_types, fontweight="bold" ,minor=False, rotation = 90, ha = 'center', fontsize = x_tick_fontsize)
                        #ax1.set_yticks(yticks, minor=False)
                        #ax1.set_ylim(yticks[0], yticks[-1])
                        axs[2*j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                        site_name_count += 1
                else: #do not use ticker.
                    if '/' in sites_for_plot[site_name_count]: #multi-sites.
                        #pie
                        axs[2*j+1].pie(pie_size[i][j], radius=PIE_R, colors = multi_colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})
                        axs[2*j+1].pie(pie_size[i][j], radius=PIE_R, colors = multi_colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})[0][1].set_hatch(HATCH)
                        #bar
                        axs[i, 2*j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                        axs[i, 2*j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                        #yticks = ticker.MaxNLocator(6)
                        #axs[i, 2*j].yaxis.set_major_locator(yticks)
                        for axis in ["left", 'bottom']:
                            axs[2*j].spines[axis].set_linewidth(spine_width)
                        for spine in ["top", "right"]:
                            axs[2*j].spines[spine].set_visible(False)
                        index = [i for i in range(3)]
                        xticks = np.arange(3)
                        yticks = np.arange(max(bar_size[i][j])+1)
                        axs[2*j].bar(index[0], bar_size[i][j][0], multi_bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                        axs[2*j].bar(index[1], bar_size[i][j][1], multi_bar_width, color=['#ffc0cb'], edgecolor='black', linewidth = bar_line_w, hatch=HATCH)
                        axs[2*j].bar(index[2], bar_size[i][j][2], multi_bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                        axs[2*j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                        axs[2*j].title.set_position([-0.15, 1.1])
                        axs[2*j].set_xticks(xticks, minor=False)
                        axs[2*j].set_xticklabels(multi_bar_types , fontweight="bold" ,minor=False, rotation = 45, ha = 'right', fontsize = x_tick_fontsize)
                        axs[2*j].set_yticks(yticks, minor=False)
                        axs[2*j].set_ylim(yticks[0], yticks[-1])
                        axs[2*j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                        bars = [rect for rect in axs[2*j].get_children() if isinstance(rect, mpl.patches.Rectangle)]
                        print('this is the length of bars: %s'%len(bars))
                        bar_type_count = 0
                        multi_bar_labels = ['HM', 'H/C', 'UN']
                        for bar in bars:
                            if bar_type_count != 3: 
                                height = bar.get_height() + MULTI_SITE_BAR_LABEL_PAD
                                label_x_pos = bar.get_x() + bar.get_width() / 2
                                axs[2*j].text(label_x_pos, height, s=multi_bar_labels[bar_type_count], ha='center',
                                va='bottom', fontsize = MULTI_SITE_BAR_FONT_SIZE)
                                bar_type_count += 1
                            else:
                                pass
                        site_name_count += 1
                    else: #real single sites.
                        print('this is i,j: %s,%s'%(i,j))
                        #pie
                        axs[2*j+1].pie(pie_size[i][j], radius=PIE_R, colors = colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})
                        #bar
                        axs[2*j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                        axs[2*j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                        #yticks = ticker.MaxNLocator(6)
                        #axs[i, 0].yaxis.set_major_locator(yticks)
                        for axis in ["left", 'bottom']:
                            axs[2*j].spines[axis].set_linewidth(spine_width)
                        for spine in ["top", "right"]:
                            axs[2*j].spines[spine].set_visible(False)
                        index = [i for i in range(16)]
                        xticks = np.arange(16)
                        yticks = np.arange(max(bar_size[i][j])+1)
                        axs[2*j].bar(index[0:5], bar_size[i][j][0:5], bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                        axs[2*j].bar(index[5:7], bar_size[i][j][5:7], bar_width, color=['#ffffff'], edgecolor='black', linewidth = bar_line_w)
                        axs[2*j].bar(index[7:15], bar_size[i][j][7:15], bar_width, color=['#ff69b4'], edgecolor='black', linewidth = bar_line_w)
                        axs[2*j].bar(index[15], bar_size[i][j][15], bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                        axs[2*j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                        axs[2*j].title.set_position([-0.15, 1.1])
                        axs[2*j].set_xticks(xticks, minor=False)
                        axs[2*j].set_xticklabels(bar_types , fontweight="bold" ,minor=False, rotation = 90, ha = 'center', fontsize = x_tick_fontsize)
                        axs[2*j].set_yticks(yticks, minor=False)
                        axs[2*j].set_ylim(yticks[0], yticks[-1])
                        axs[2*j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                        site_name_count += 1
            else: #empty subplot(s). hide spines & ticks & labels.
                print('this is i,j for empty subplots: %s,%s'%(i,j))
                axs[2*j].spines["left"].set_visible(False)
                axs[2*j].spines["right"].set_visible(False)
                axs[2*j].spines["top"].set_visible(False)
                axs[2*j].spines["bottom"].set_visible(False)
                axs[2*j+1].spines["left"].set_visible(False)
                axs[2*j+1].spines["right"].set_visible(False)
                axs[2*j+1].spines["top"].set_visible(False)
                axs[2*j+1].spines["bottom"].set_visible(False)
                axs[2*j].xaxis.set_ticks([])
                axs[2*j].yaxis.set_ticks([])
                axs[2*j+1].xaxis.set_ticks([])
                axs[2*j+1].yaxis.set_ticks([])


    params = {'legend.fontsize': legend_fontsize, 'legend.handlelength': legend_handleL}
    legend_properties = {'weight':'bold'}
    plt.rcParams.update(params)

    legend_elements = []
    for i in range(len(leg_colors)):
        if i == 3:
            each_line = Patch(facecolor = leg_colors[i], edgecolor = 'k', label = pie_types[i], lw = 1.5, hatch = HATCH)
            legend_elements.append(each_line)
        else:
            each_line = Patch(facecolor = leg_colors[i], edgecolor = 'k', label = pie_types[i], lw = 1.5)
            legend_elements.append(each_line)


    plt.legend(handles=legend_elements, prop=legend_properties, bbox_to_anchor=(1.2,-0.2), loc='upper left')

    #plt.tight_layout()
    print('\nExport Pie & Bar charts as .png files.')
    fig.savefig(f'{date}_{filename}_{software}BarPieCharts.png')

    ## DRAW ALL PIE CHARTS (if site num > 4).
    if len(original_pie_size) > 4: 
    ##### adjustable parameters ###################
    #if len(original_pie_size) < 4:
    #    number_of_sites_inrow = 1
    #    full_row_num = len(original_pie_size)//number_of_sites_inrow
    #    remainder_num = len(original_pie_size)%number_of_sites_inrow
    #    pie_remainder = original_pie_size[-remainder_num:] #last ones.
    #    print('pie_remainder: %s'%pie_remainder)
    #    pie_size = [original_pie_size[number_of_sites_inrow*i:number_of_sites_inrow*(i+1)] for i in range(full_row_num)]
    #    print('this is resized pie_size: %s'%pie_size)
    #else:
        number_of_sites_inrow = 4
        full_row_num = len(original_pie_size)//number_of_sites_inrow
        remainder_num = len(original_pie_size)%number_of_sites_inrow
        pie_remainder = original_pie_size[-remainder_num:] #last ones.
        print('pie_remainder: %s'%pie_remainder)
        pie_size = [original_pie_size[number_of_sites_inrow*i:number_of_sites_inrow*(i+1)] for i in range(full_row_num)]
        pie_size.append(pie_remainder)
        print('this is resized pie_size: %s'%pie_size)

        if remainder_num == 0:
            row_num = full_row_num
        else: #e.g. 3 sites in total.
            row_num = full_row_num + 1

        print('this is row_num: %s'%row_num)
        print('this is remainder_num: %s'%remainder_num)
        col_num = 1*number_of_sites_inrow
        fig_ratio = number_of_sites_inrow/row_num
        ###############################################

        #make everything in the plot bold & basic setups.
        ####### fontsize & spine width #######
        pie_title_fontsize = 12
        SUBPLOT_PAD = 20
        pie_line_w = 1.2
        legend_fontsize = bar_title_fontsize
        legend_handleL = 2
        PIE_R = 1.75
        HATCH = '/'*4
        PIE_PLOTS_RATIO_1 = 2
        PIE_PLOTS_RATIO_2 = 2
        ######################################
        fig, axs = plt.subplots(nrows=row_num, ncols=col_num, figsize=(PIE_PLOTS_RATIO_1*number_of_sites_inrow, PIE_PLOTS_RATIO_2*row_num), constrained_layout=False)
        site_name_count = 0

        if row_num >1: 
            for i in range(row_num):
                if i != (row_num -1): #other than the last row. 
                    for j in range(number_of_sites_inrow): #N sites from left to right.
                        if '/' in sites_for_plot[site_name_count]: #multi-sites.
                            #pie
                            axs[i, j].pie(pie_size[i][j], radius=PIE_R, colors = multi_colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})
                            axs[i, j].pie(pie_size[i][j], radius=PIE_R, colors = multi_colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})[0][1].set_hatch(HATCH)
                            axs[i, j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                            site_name_count += 1
                        else: #real singel sites.
                            #pie
                            axs[i, j].pie(pie_size[i][j], radius=PIE_R, colors = colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})
                            axs[i, j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                            site_name_count += 1
                else: #for the last row. Deal with the remainder.
                    if remainder_num == 0: #can be divided by 8.
                        last_row_site_num = number_of_sites_inrow
                    else: #cannot be divided by 8.
                        last_row_site_num = remainder_num
                    print('this is last_row_site_num: %s'%last_row_site_num)
                    for j in range(number_of_sites_inrow):
                        if j < last_row_site_num:  
                            print('this is i,j: %s,%s'%(i,j))
                            if '/' in sites_for_plot[site_name_count]: #multi-sites.
                                #pie
                                axs[i, j].pie(pie_size[i][j], radius=PIE_R, colors = multi_colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})
                                axs[i, j].pie(pie_size[i][j], radius=PIE_R, colors = multi_colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})[0][1].set_hatch(HATCH)
                                axs[i, j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                                site_name_count += 1
                            else: #real single sites.
                                #pie
                                axs[i, j].pie(pie_size[i][j], radius=PIE_R, colors = colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})
                                axs[i, j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                                site_name_count += 1       
                        else: #empty subplot(s). hide spines & ticks & labels.
                            print('this is i,j for empty subplots: %s,%s'%(i,j))
                            axs[i, j].spines["left"].set_visible(False)
                            axs[i, j].spines["right"].set_visible(False)
                            axs[i, j].spines["top"].set_visible(False)
                            axs[i, j].spines["bottom"].set_visible(False)
                            axs[i, j].xaxis.set_ticks([])
                            axs[i, j].yaxis.set_ticks([])
        else: #if only one row. (based on the layout setting above, this will happen if there's only one site.)
            i = 0
            if remainder_num == 0: #can be divided by 8.
                last_row_site_num = number_of_sites_inrow
            else: #cannot be divided by 8.
                last_row_site_num = remainder_num
            print('this is last_row_site_num: %s'%last_row_site_num)
            for j in range(number_of_sites_inrow):
                if j < last_row_site_num:
                    if '/' in sites_for_plot[site_name_count]: #multi-sites.
                        #pie
                        axs[j].pie(pie_size[i][j], radius=PIE_R, colors = multi_colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})
                        axs[j].pie(pie_size[i][j], radius=PIE_R, colors = multi_colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})[0][1].set_hatch(HATCH)
                        axs[j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                        site_name_count += 1
                    else: #real single sites.
                        #pie
                        axs[j].pie(pie_size[i][j], radius=PIE_R, colors = colors, shadow = True, wedgeprops={"edgecolor":"k",'linewidth': pie_line_w, 'antialiased': True})
                        axs[j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                        site_name_count += 1
                else: #empty subplot(s). hide spines & ticks & labels.
                    axs[j].spines["left"].set_visible(False)
                    axs[j].spines["right"].set_visible(False)
                    axs[j].spines["top"].set_visible(False)
                    axs[j].spines["bottom"].set_visible(False)
                    axs[j].xaxis.set_ticks([])
                    axs[j].yaxis.set_ticks([])

        params = {'legend.fontsize': legend_fontsize, 'legend.handlelength': legend_handleL}
        legend_properties = {'weight':'bold'}
        plt.rcParams.update(params)

        legend_elements = []
        for i in range(len(leg_colors)):
            if i == 3:
                each_line = Patch(facecolor = leg_colors[i], edgecolor = 'k', label = pie_types[i], lw = 1.5, hatch = HATCH)
                legend_elements.append(each_line)
            else:
                each_line = Patch(facecolor = leg_colors[i], edgecolor = 'k', label = pie_types[i], lw = 1.5)
                legend_elements.append(each_line)

        plt.legend(handles=legend_elements, prop=legend_properties, bbox_to_anchor=(1.2,-0.2), loc='upper left')

        plt.tight_layout()
        print('\nExport All Pie charts as .png')
        fig.savefig(f'{date}_{filename}_{software}PieCharts.png')


        ## DRAW ALL BAR CHARTS (if site num > 4). 
        ##### adjustable parameters ###################
        #if len(original_bar_size) < 4:
        #    number_of_sites_inrow = 1
        #    full_row_num = len(original_bar_size)//number_of_sites_inrow
        #    remainder_num = len(original_bar_size)%number_of_sites_inrow
        #    bar_remainder = original_bar_size[-remainder_num:] #last ones.
        #    print('bar_remainder: %s'%bar_remainder)
        #    bar_size = [original_bar_size[number_of_sites_inrow*i:number_of_sites_inrow*(i+1)] for i in range(full_row_num)]
        #    print('this is resized bar_size: %s'%bar_size)
        #else:
        number_of_sites_inrow = 4
        full_row_num = len(original_bar_size)//number_of_sites_inrow
        remainder_num = len(original_bar_size)%number_of_sites_inrow
        bar_remainder = original_bar_size[-remainder_num:] #last ones.
        print('bar_remainder: %s'%bar_remainder)
        bar_size = [original_bar_size[number_of_sites_inrow*i:number_of_sites_inrow*(i+1)] for i in range(full_row_num)]
        bar_size.append(bar_remainder)
        print('this is resized bar_size: %s'%bar_size)

        if remainder_num == 0:
            row_num = full_row_num
        else: #e.g. 3 sites in total.
            row_num = full_row_num + 1

        print('this is row_num: %s'%row_num)
        print('this is remainder_num: %s'%remainder_num)
        col_num = 1*number_of_sites_inrow
        fig_ratio = number_of_sites_inrow/row_num
        ###############################################

        #make everything in the plot bold & basic setups.
        ####### fontsize & spine width #######
        bar_title_fontsize = 15
        SUBPLOT_PAD = 6
        bar_ylab_fontsize = bar_title_fontsize - 4
        bar_width = 0.7
        multi_bar_width = 0.1
        x_tick_fontsize = bar_title_fontsize -4 
        y_tick_fonsize = bar_title_fontsize -4
        x_tick_w = 1.2
        x_tick_l = x_tick_w*3
        y_tick_w = 1.2
        y_tick_l = y_tick_w*3
        spine_width = 1.2
        pie_line_w = 1.2
        legend_fontsize = bar_title_fontsize
        legend_handleL = 2
        MULTI_SITE_BAR_FONT_SIZE = 8
        MULTI_SITE_BAR_LABEL_PAD = 0
        PSM_LABEL_PAD = 1.5
        PIE_R = 1.75
        HATCH = '////'
        TICKER_THRESHOLD = 6
        MAXNLOCATOR_NBINS = 5
        BAR_PLOTS_RATIO_1 = 5
        BAR_PLOTS_RATIO_2 = 2.5
        ######################################
        plt.rcParams["font.weight"] = "bold"
        plt.rcParams["axes.labelweight"] = "bold"
        fig, axs = plt.subplots(nrows=row_num, ncols=col_num, figsize=(BAR_PLOTS_RATIO_1*number_of_sites_inrow, BAR_PLOTS_RATIO_2*row_num), constrained_layout=False)
        site_name_count = 0

        if row_num >1: 
            for i in range(row_num):
                if i != (row_num -1): #other than the last row. 
                    for j in range(number_of_sites_inrow): #N sites from left to right.
                        if max(bar_size[i][j]) > TICKER_THRESHOLD: #use ticker.
                            if '/' in sites_for_plot[site_name_count]: #multi-sites.
                                #bar
                                axs[i, j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                                axs[i, j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                                yticks = ticker.MaxNLocator(MAXNLOCATOR_NBINS, integer = True)
                                axs[i, j].yaxis.set_major_locator(yticks)
                                for axis in ["left", 'bottom']:
                                    axs[i, j].spines[axis].set_linewidth(spine_width)
                                for spine in ["top", "right"]:
                                    axs[i, j].spines[spine].set_visible(False)
                                index = [i for i in range(3)]
                                xticks = np.arange(3)
                                #yticks = np.arange(max(size[i])+1)
                                axs[i, j].bar(index[0], bar_size[i][j][0], multi_bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, j].bar(index[1], bar_size[i][j][1], multi_bar_width, color=['#ffc0cb'], edgecolor='black', linewidth = bar_line_w, hatch=HATCH)
                                axs[i, j].bar(index[2], bar_size[i][j][2], multi_bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                                axs[i, j].title.set_position([-0.15, 1.1])
                                axs[i, j].set_xticks(xticks, minor=False)
                                axs[i, j].set_xticklabels('')
                                #axs[i, 2*j].set_xticklabels(multi_bar_types , fontweight="bold" ,minor=False, rotation = 45, ha = 'center', fontsize = x_tick_fontsize)
                                #ax1.set_yticks(yticks, minor=False)
                                #ax1.set_ylim(yticks[0], yticks[-1])
                                axs[i, j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                                bars = [rect for rect in axs[i, j].get_children() if isinstance(rect, mpl.patches.Rectangle)]
                                print('this is the length of bars: %s'%len(bars))
                                bar_type_count = 0
                                multi_bar_labels = ['HM', 'H/C', 'UN']
                                for bar in bars:
                                    if bar_type_count != 3: 
                                        height = bar.get_height() + MULTI_SITE_BAR_LABEL_PAD
                                        label_x_pos = bar.get_x() + bar.get_width() / 2
                                        axs[i, j].text(label_x_pos, height, s=multi_bar_labels[bar_type_count], ha='center',
                                        va='bottom', fontsize = MULTI_SITE_BAR_FONT_SIZE)
                                        bar_type_count += 1
                                    else:
                                        pass
                                site_name_count += 1
                            else: #real singel sites.
                                #bar
                                axs[i, j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                                axs[i, j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                                yticks = ticker.MaxNLocator(MAXNLOCATOR_NBINS, integer = True)
                                axs[i, j].yaxis.set_major_locator(yticks)
                                for axis in ["left", 'bottom']:
                                    axs[i, j].spines[axis].set_linewidth(spine_width)
                                for spine in ["top", "right"]:
                                    axs[i, j].spines[spine].set_visible(False)
                                index = [i for i in range(16)]
                                xticks = np.arange(16)
                                #yticks = np.arange(max(size[i])+1)
                                axs[i, j].bar(index[0:5], bar_size[i][j][0:5], bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, j].bar(index[5:7], bar_size[i][j][5:7], bar_width, color=['#ffffff'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, j].bar(index[7:15], bar_size[i][j][7:15], bar_width, color=['#ff69b4'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, j].bar(index[15], bar_size[i][j][15], bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                                axs[i, j].title.set_position([-0.15, 1.1])
                                axs[i, j].set_xticks(xticks, minor=False)
                                axs[i, j].set_xticklabels('')
                                #ax1.set_yticks(yticks, minor=False)
                                #ax1.set_ylim(yticks[0], yticks[-1])
                                axs[i, j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                                site_name_count += 1
                        else: #do not use ticker.
                            if '/' in sites_for_plot[site_name_count]: #multi-sites.
                                #bar
                                axs[i, j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                                axs[i, j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                                #yticks = ticker.MaxNLocator(6)
                                #axs[i, 2*j].yaxis.set_major_locator(yticks)
                                for axis in ["left", 'bottom']:
                                    axs[i, j].spines[axis].set_linewidth(spine_width)
                                for spine in ["top", "right"]:
                                    axs[i, j].spines[spine].set_visible(False)
                                index = [i for i in range(3)]
                                xticks = np.arange(3)
                                yticks = np.arange(max(bar_size[i][j])+1)
                                axs[i, j].bar(index[0], bar_size[i][j][0], multi_bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, j].bar(index[1], bar_size[i][j][1], multi_bar_width, color=['#ffc0cb'], edgecolor='black', linewidth = bar_line_w, hatch=HATCH)
                                axs[i, j].bar(index[2], bar_size[i][j][2], multi_bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                                axs[i, j].title.set_position([-0.15, 1.1])
                                axs[i, j].set_xticks(xticks, minor=False)
                                axs[i, j].set_xticklabels('')
                                #axs[i, 2*j].set_xticklabels(multi_bar_types , fontweight="bold" ,minor=False, rotation = 45, ha = 'center', fontsize = x_tick_fontsize)
                                axs[i, j].set_yticks(yticks, minor=False)
                                axs[i, j].set_ylim(yticks[0], yticks[-1])
                                axs[i, j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                                bars = [rect for rect in axs[i, j].get_children() if isinstance(rect, mpl.patches.Rectangle)]
                                print('this is the length of bars: %s'%len(bars))
                                bar_type_count = 0
                                multi_bar_labels = ['HM', 'H/C', 'UN']
                                for bar in bars:
                                    if bar_type_count != 3: 
                                        height = bar.get_height() + MULTI_SITE_BAR_LABEL_PAD 
                                        label_x_pos = bar.get_x() + bar.get_width() / 2
                                        axs[i, j].text(label_x_pos, height, s=multi_bar_labels[bar_type_count], ha='center',
                                        va='bottom', fontsize = MULTI_SITE_BAR_FONT_SIZE)
                                        bar_type_count += 1
                                    else:
                                        pass
                                site_name_count += 1
                            else: #real singel sites.
                                #bar
                                axs[i, j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                                axs[i, j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                                #yticks = ticker.MaxNLocator(6)
                                #axs[i, 2*j].yaxis.set_major_locator(yticks)
                                for axis in ["left", 'bottom']:
                                    axs[i, j].spines[axis].set_linewidth(spine_width)
                                for spine in ["top", "right"]:
                                    axs[i, j].spines[spine].set_visible(False)
                                index = [i for i in range(16)]
                                xticks = np.arange(16)
                                yticks = np.arange(max(bar_size[i][j])+1)
                                axs[i, j].bar(index[0:5], bar_size[i][j][0:5], bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, j].bar(index[5:7], bar_size[i][j][5:7], bar_width, color=['#ffffff'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, j].bar(index[7:15], bar_size[i][j][7:15], bar_width, color=['#ff69b4'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, j].bar(index[15], bar_size[i][j][15], bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                                axs[i, j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                                axs[i, j].title.set_position([-0.15, 1.1])
                                axs[i, j].set_xticks(xticks, minor=False)
                                axs[i, j].set_xticklabels('')
                                axs[i, j].set_yticks(yticks, minor=False)
                                axs[i, j].set_ylim(yticks[0], yticks[-1])
                                axs[i, j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                                site_name_count += 1
                else: #for the last row. Deal with the remainder.
                    if remainder_num == 0: #can be divided by 8.
                        last_row_site_num = number_of_sites_inrow
                    else: #cannot be divided by 8.
                        last_row_site_num = remainder_num
                    print('this is last_row_site_num: %s'%last_row_site_num)
                    for j in range(number_of_sites_inrow):
                        if j < last_row_site_num:  
                            print('this is i,j: %s,%s'%(i,j))
                            if max(bar_size[i][j]) > TICKER_THRESHOLD: #use ticker.
                                if '/' in sites_for_plot[site_name_count]: #multi-sites.
                                    #bar
                                    axs[i, j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                                    axs[i, j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                                    yticks = ticker.MaxNLocator(MAXNLOCATOR_NBINS, integer = True)
                                    axs[i, j].yaxis.set_major_locator(yticks)
                                    for axis in ["left", 'bottom']:
                                        axs[i, j].spines[axis].set_linewidth(spine_width)
                                    for spine in ["top", "right"]:
                                        axs[i, j].spines[spine].set_visible(False)
                                    index = [i for i in range(3)]
                                    xticks = np.arange(3)
                                    #yticks = np.arange(max(size[i])+1)
                                    axs[i, j].bar(index[0], bar_size[i][j][0], multi_bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                                    axs[i, j].bar(index[1], bar_size[i][j][1], multi_bar_width, color=['#ffc0cb'], edgecolor='black', linewidth = bar_line_w, hatch=HATCH)
                                    axs[i, j].bar(index[2], bar_size[i][j][2], multi_bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                                    axs[i, j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                                    axs[i, j].title.set_position([-0.15, 1.1])
                                    axs[i, j].set_xticks(xticks, minor=False)
                                    axs[i, j].set_xticklabels(multi_bar_types , fontweight="bold" ,minor=False, rotation = 45, ha = 'right', fontsize = x_tick_fontsize)
                                    #ax1.set_yticks(yticks, minor=False)
                                    #ax1.set_ylim(yticks[0], yticks[-1])
                                    axs[i, j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                                    bars = [rect for rect in axs[i, 2*j].get_children() if isinstance(rect, mpl.patches.Rectangle)]
                                    print('this is the length of bars: %s'%len(bars))
                                    bar_type_count = 0
                                    multi_bar_labels = ['HM', 'H/C', 'UN']
                                    for bar in bars:
                                        if bar_type_count != 3: 
                                            height = bar.get_height() + MULTI_SITE_BAR_LABEL_PAD
                                            label_x_pos = bar.get_x() + bar.get_width() / 2
                                            axs[i, j].text(label_x_pos, height, s=multi_bar_labels[bar_type_count], ha='center',
                                            va='bottom', fontsize = MULTI_SITE_BAR_FONT_SIZE)
                                            bar_type_count += 1
                                        else:
                                            pass
                                    site_name_count += 1
                                else: #real single sites.
                                    #bar
                                    axs[i, j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                                    axs[i, j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                                    yticks = ticker.MaxNLocator(MAXNLOCATOR_NBINS, integer = True)
                                    axs[i, j].yaxis.set_major_locator(yticks)
                                    for axis in ["left", 'bottom']:
                                        axs[i, j].spines[axis].set_linewidth(spine_width)
                                    for spine in ["top", "right"]:
                                        axs[i, j].spines[spine].set_visible(False)
                                    index = [i for i in range(16)]
                                    xticks = np.arange(16)
                                    #yticks = np.arange(max(size[i])+1)
                                    axs[i, j].bar(index[0:5], bar_size[i][j][0:5], bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                                    axs[i, j].bar(index[5:7], bar_size[i][j][5:7], bar_width, color=['#ffffff'], edgecolor='black', linewidth = bar_line_w)
                                    axs[i, j].bar(index[7:15], bar_size[i][j][7:15], bar_width, color=['#ff69b4'], edgecolor='black', linewidth = bar_line_w)
                                    axs[i, j].bar(index[15], bar_size[i][j][15], bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                                    axs[i, j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                                    axs[i, j].title.set_position([-0.15, 1.1])
                                    axs[i, j].set_xticks(xticks, minor=False)
                                    axs[i, j].set_xticklabels( bar_types, fontweight="bold" ,minor=False, rotation = 90, ha = 'center', fontsize = x_tick_fontsize)
                                    #ax1.set_yticks(yticks, minor=False)
                                    #ax1.set_ylim(yticks[0], yticks[-1])
                                    axs[i, j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                                    site_name_count += 1
                            else: #do not use ticker.
                                if '/' in sites_for_plot[site_name_count]: #multi-sites.
                                    #bar
                                    axs[i, j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                                    axs[i, j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                                    #yticks = ticker.MaxNLocator(6)
                                    #axs[i, 2*j].yaxis.set_major_locator(yticks)
                                    for axis in ["left", 'bottom']:
                                        axs[i, j].spines[axis].set_linewidth(spine_width)
                                    for spine in ["top", "right"]:
                                        axs[i, j].spines[spine].set_visible(False)
                                    index = [i for i in range(3)]
                                    xticks = np.arange(3)
                                    yticks = np.arange(max(bar_size[i][j])+1)
                                    axs[i, j].bar(index[0], bar_size[i][j][0], multi_bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                                    axs[i, j].bar(index[1], bar_size[i][j][1], multi_bar_width, color=['#ffc0cb'], edgecolor='black', linewidth = bar_line_w, hatch=HATCH)
                                    axs[i, j].bar(index[2], bar_size[i][j][2], multi_bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                                    axs[i, j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                                    axs[i, j].title.set_position([-0.15, 1.1])
                                    axs[i, j].set_xticks(xticks, minor=False)
                                    axs[i, j].set_xticklabels(multi_bar_types , fontweight="bold" ,minor=False, rotation = 45, ha = 'right', fontsize = x_tick_fontsize)
                                    axs[i, j].set_yticks(yticks, minor=False)
                                    axs[i, j].set_ylim(yticks[0], yticks[-1])
                                    axs[i, j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                                    bars = [rect for rect in axs[i, j].get_children() if isinstance(rect, mpl.patches.Rectangle)]
                                    print('this is the length of bars: %s'%len(bars))
                                    bar_type_count = 0
                                    multi_bar_labels = ['HM', 'H/C', 'UN']
                                    for bar in bars:
                                        if bar_type_count != 3: 
                                            height = bar.get_height() + MULTI_SITE_BAR_LABEL_PAD
                                            label_x_pos = bar.get_x() + bar.get_width() / 2
                                            axs[i, j].text(label_x_pos, height, s=multi_bar_labels[bar_type_count], ha='center',
                                            va='bottom', fontsize = MULTI_SITE_BAR_FONT_SIZE)
                                            bar_type_count += 1
                                        else:
                                            pass
                                    site_name_count += 1
                                else: #real single sites.
                                    print('this is i,j: %s,%s'%(i,j))
                                    #bar
                                    axs[i, j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                                    axs[i, j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                                    #yticks = ticker.MaxNLocator(6)
                                    #axs[i, 0].yaxis.set_major_locator(yticks)
                                    for axis in ["left", 'bottom']:
                                        axs[i, j].spines[axis].set_linewidth(spine_width)
                                    for spine in ["top", "right"]:
                                        axs[i, j].spines[spine].set_visible(False)
                                    index = [i for i in range(16)]
                                    xticks = np.arange(16)
                                    yticks = np.arange(max(bar_size[i][j])+1)
                                    axs[i, j].bar(index[0:5], bar_size[i][j][0:5], bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                                    axs[i, j].bar(index[5:7], bar_size[i][j][5:7], bar_width, color=['#ffffff'], edgecolor='black', linewidth = bar_line_w)
                                    axs[i, j].bar(index[7:15], bar_size[i][j][7:15], bar_width, color=['#ff69b4'], edgecolor='black', linewidth = bar_line_w)
                                    axs[i, j].bar(index[15], bar_size[i][j][15], bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                                    axs[i, j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                                    axs[i, j].title.set_position([-0.15, 1.1])
                                    axs[i, j].set_xticks(xticks, minor=False)
                                    axs[i, j].set_xticklabels(bar_types , fontweight="bold" ,minor=False, rotation = 90, ha = 'center', fontsize = x_tick_fontsize)
                                    axs[i, j].set_yticks(yticks, minor=False)
                                    axs[i, j].set_ylim(yticks[0], yticks[-1])
                                    axs[i, j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                                    site_name_count += 1
                        else: #empty subplot(s). hide spines & ticks & labels.
                            print('this is i,j for empty subplots: %s,%s'%(i,j))
                            axs[i, j].spines["left"].set_visible(False)
                            axs[i, j].spines["right"].set_visible(False)
                            axs[i, j].spines["top"].set_visible(False)
                            axs[i, j].spines["bottom"].set_visible(False)
                            axs[i, j].xaxis.set_ticks([])
                            axs[i, j].yaxis.set_ticks([])
        else: #if only one row. (based on the layout setting above, this will happen if there's only one site.)
            i = 0
            if remainder_num == 0: #can be divided by 4.
                last_row_site_num = number_of_sites_inrow
            else: #cannot be divided by 4.
                last_row_site_num = remainder_num
            print('this is last_row_site_num: %s'%last_row_site_num)
            for j in range(number_of_sites_inrow):
                if j < last_row_site_num:  
                    if max(bar_size[i][j]) > TICKER_THRESHOLD: #use ticker.
                        if '/' in sites_for_plot[site_name_count]: #multi-sites.
                            #bar
                            axs[j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                            axs[j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                            yticks = ticker.MaxNLocator(MAXNLOCATOR_NBINS, integer = True)
                            axs[j].yaxis.set_major_locator(yticks)
                            for axis in ["left", 'bottom']:
                                axs[j].spines[axis].set_linewidth(spine_width)
                            for spine in ["top", "right"]:
                                axs[j].spines[spine].set_visible(False)
                            index = [i for i in range(3)]
                            xticks = np.arange(3)
                            #yticks = np.arange(max(size[i])+1)
                            axs[j].bar(index[0], bar_size[i][j][0], multi_bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                            axs[j].bar(index[1], bar_size[i][j][1], multi_bar_width, color=['#ffc0cb'], edgecolor='black', linewidth = bar_line_w, hatch=HATCH)
                            axs[j].bar(index[2], bar_size[i][j][2], multi_bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                            axs[j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                            axs[j].title.set_position([-0.15, 1.1])
                            axs[j].set_xticks(xticks, minor=False)
                            axs[j].set_xticklabels(multi_bar_types , fontweight="bold" ,minor=False, rotation = 45, ha = 'right', fontsize = x_tick_fontsize)
                            #ax1.set_yticks(yticks, minor=False)
                            #ax1.set_ylim(yticks[0], yticks[-1])
                            axs[j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                            bars = [rect for rect in axs[j].get_children() if isinstance(rect, mpl.patches.Rectangle)]
                            print('this is the length of bars: %s'%len(bars))
                            bar_type_count = 0
                            multi_bar_labels = ['HM', 'H/C', 'UN']
                            for bar in bars:
                                if bar_type_count != 3: 
                                    height = bar.get_height() + MULTI_SITE_BAR_LABEL_PAD
                                    label_x_pos = bar.get_x() + bar.get_width() / 2
                                    axs[j].text(label_x_pos, height, s=multi_bar_labels[bar_type_count], ha='center',
                                    va='bottom', fontsize = MULTI_SITE_BAR_FONT_SIZE)
                                    bar_type_count += 1
                                else:
                                    pass
                            site_name_count += 1
                        else: #real single sites.
                            #bar
                            axs[j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                            axs[j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                            yticks = ticker.MaxNLocator(MAXNLOCATOR_NBINS, integer = True)
                            axs[j].yaxis.set_major_locator(yticks)
                            for axis in ["left", 'bottom']:
                                axs[j].spines[axis].set_linewidth(spine_width)
                            for spine in ["top", "right"]:
                                axs[j].spines[spine].set_visible(False)
                            index = [i for i in range(16)]
                            xticks = np.arange(16)
                            #yticks = np.arange(max(size[i])+1)
                            axs[j].bar(index[0:5], bar_size[i][j][0:5], bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                            axs[j].bar(index[5:7], bar_size[i][j][5:7], bar_width, color=['#ffffff'], edgecolor='black', linewidth = bar_line_w)
                            axs[j].bar(index[7:15], bar_size[i][j][7:15], bar_width, color=['#ff69b4'], edgecolor='black', linewidth = bar_line_w)
                            axs[j].bar(index[15], bar_size[i][j][15], bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                            axs[j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                            axs[j].title.set_position([-0.15, 1.1])
                            axs[j].set_xticks(xticks, minor=False)
                            axs[j].set_xticklabels( bar_types, fontweight="bold" ,minor=False, rotation = 90, ha = 'center', fontsize = x_tick_fontsize)
                            #ax1.set_yticks(yticks, minor=False)
                            #ax1.set_ylim(yticks[0], yticks[-1])
                            axs[j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                            site_name_count += 1
                    else: #do not use ticker.
                        if '/' in sites_for_plot[site_name_count]: #multi-sites.
                            #bar
                            axs[j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                            axs[j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                            #yticks = ticker.MaxNLocator(6)
                            #axs[i, 2*j].yaxis.set_major_locator(yticks)
                            for axis in ["left", 'bottom']:
                                axs[j].spines[axis].set_linewidth(spine_width)
                            for spine in ["top", "right"]:
                                axs[j].spines[spine].set_visible(False)
                            index = [i for i in range(3)]
                            xticks = np.arange(3)
                            yticks = np.arange(max(bar_size[i][j])+1)
                            axs[j].bar(index[0], bar_size[i][j][0], multi_bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                            axs[j].bar(index[1], bar_size[i][j][1], multi_bar_width, color=['#ffc0cb'], edgecolor='black', linewidth = bar_line_w, hatch=HATCH)
                            axs[j].bar(index[2], bar_size[i][j][2], multi_bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                            axs[j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                            axs[j].title.set_position([-0.15, 1.1])
                            axs[j].set_xticks(xticks, minor=False)
                            axs[j].set_xticklabels(multi_bar_types , fontweight="bold" ,minor=False, rotation = 45, ha = 'right', fontsize = x_tick_fontsize)
                            axs[j].set_yticks(yticks, minor=False)
                            axs[j].set_ylim(yticks[0], yticks[-1])
                            axs[j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                            bars = [rect for rect in axs[j].get_children() if isinstance(rect, mpl.patches.Rectangle)]
                            print('this is the length of bars: %s'%len(bars))
                            bar_type_count = 0
                            multi_bar_labels = ['HM', 'H/C', 'UN']
                            for bar in bars:
                                if bar_type_count != 3: 
                                    height = bar.get_height() + MULTI_SITE_BAR_LABEL_PAD
                                    label_x_pos = bar.get_x() + bar.get_width() / 2
                                    axs[j].text(label_x_pos, height, s=multi_bar_labels[bar_type_count], ha='center',
                                    va='bottom', fontsize = MULTI_SITE_BAR_FONT_SIZE)
                                    bar_type_count += 1
                                else:
                                    pass
                            site_name_count += 1
                        else: #real single sites.
                            print('this is i,j: %s,%s'%(i,j))
                            #bar
                            axs[j].xaxis.set_tick_params(width=x_tick_w, length=x_tick_l)
                            axs[j].yaxis.set_tick_params(width=y_tick_w, length=y_tick_l, labelsize = y_tick_fonsize)
                            #yticks = ticker.MaxNLocator(6)
                            #axs[i, 0].yaxis.set_major_locator(yticks)
                            for axis in ["left", 'bottom']:
                                axs[j].spines[axis].set_linewidth(spine_width)
                            for spine in ["top", "right"]:
                                axs[j].spines[spine].set_visible(False)
                            index = [i for i in range(16)]
                            xticks = np.arange(16)
                            yticks = np.arange(max(bar_size[i][j])+1)
                            axs[j].bar(index[0:5], bar_size[i][j][0:5], bar_width, color=['#008000'], edgecolor='black', linewidth = bar_line_w)
                            axs[j].bar(index[5:7], bar_size[i][j][5:7], bar_width, color=['#ffffff'], edgecolor='black', linewidth = bar_line_w)
                            axs[j].bar(index[7:15], bar_size[i][j][7:15], bar_width, color=['#ff69b4'], edgecolor='black', linewidth = bar_line_w)
                            axs[j].bar(index[15], bar_size[i][j][15], bar_width, color=['#d3d3d3'], edgecolor='black', linewidth = bar_line_w)
                            axs[j].set_title(sites_for_plot[site_name_count], size = bar_title_fontsize, fontweight="bold", pad = SUBPLOT_PAD, ha=bar_title_ha)
                            axs[j].title.set_position([-0.15, 1.1])
                            axs[j].set_xticks(xticks, minor=False)
                            axs[j].set_xticklabels(bar_types , fontweight="bold" ,minor=False, rotation = 90, ha = 'center', fontsize = x_tick_fontsize)
                            axs[j].set_yticks(yticks, minor=False)
                            axs[j].set_ylim(yticks[0], yticks[-1])
                            axs[j].set_ylabel("PSM", fontsize = bar_ylab_fontsize, fontweight="bold", labelpad = PSM_LABEL_PAD)
                            site_name_count += 1
                else: #empty subplot(s). hide spines & ticks & labels.
                    print('this is i,j for empty subplots: %s,%s'%(i,j))
                    axs[j].spines["left"].set_visible(False)
                    axs[j].spines["right"].set_visible(False)
                    axs[j].spines["top"].set_visible(False)
                    axs[j].spines["bottom"].set_visible(False)
                    axs[j].xaxis.set_ticks([])
                    axs[j].yaxis.set_ticks([])


        params = {'legend.fontsize': legend_fontsize, 'legend.handlelength': legend_handleL}
        legend_properties = {'weight':'bold'}
        plt.rcParams.update(params)

        legend_elements = []
        for i in range(len(leg_colors)):
            if i == 3:
                each_line = Patch(facecolor = leg_colors[i], edgecolor = 'k', label = pie_types[i], lw = 1.5, hatch = HATCH)
                legend_elements.append(each_line)
            else:
                each_line = Patch(facecolor = leg_colors[i], edgecolor = 'k', label = pie_types[i], lw = 1.5)
                legend_elements.append(each_line)


        plt.legend(handles=legend_elements, prop=legend_properties, bbox_to_anchor=(1.2,-0.2), loc='upper left')

        plt.tight_layout()
        print('\nExport All Bar charts as .png')
        fig.savefig(f'{date}_{filename}_{software}BarCharts.png')
    return site_nostr_new    
    
    
    